In [298]:
import numpy as np
from sklearn.cluster import DBSCAN
import re
import nibabel as nib
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

In [299]:
pathFile = open("pathFile1.txt","r")
paths = len(open("pathFile1.txt","r").readlines())
longestPathLength = int((len(max(open("pathFile1.txt","r"),key=len).split(" "))-1)/3)
print(paths)
print(longestPathLength)
dilationRadius = -1 #has to be greater than or equal to 0
numIntensityParams = ((2*(dilationRadius+1)-1)**2) #number of intensities to use
if dilationRadius <0 :
    numIntensityParams = 0
print(numIntensityParams)

1575
59
0


In [300]:
swiImage = nib.load("SWI_Images_resize10.nii")
swiImage.shape
swiData = swiImage.get_fdata()
print(swiData.shape)
#swiData = swiData/np.max(np.abs(swiData),axis=0) #normalize by the max across entire image

(80, 105, 15)


In [301]:
# start with just the xyz and intensity of the paths nothing around it yet
bunchedPaths = {}#dictionary with the keys as the root node as a joined spring and the values as a list of all paths with that root node
flatennedPaths = {}#used for initial clustering without intesities 
groupPath = [] #dyanmic array 
for lineNum, line in enumerate(pathFile):
    currentSplitLine = line.split(" ")
    currentSplitLine = currentSplitLine[:-1]
    
    currentSplitLine = list(map(float,currentSplitLine))
    gFoldedLine = np.reshape(currentSplitLine,(int(len(currentSplitLine)/3),3))
    
    
    rootNode = " ".join(list(map(str,map(int,gFoldedLine[58]))))
    gFoldedLine.astype(np.float)
    groupArray = []
    flatGroupArray = []
    if rootNode in bunchedPaths: # if the root hasnt been added to the dictionary add it to the dictionary with a list of paths
        groupArray = bunchedPaths[rootNode] #join the string to hash a string instead of a tuple since python likes that better
        flatGroupArray = flatennedPaths[rootNode]
    else:
        bunchedPaths[rootNode] = groupArray
        flatennedPaths[rootNode] =flatGroupArray 
    flatGroupArray.append(currentSplitLine)
    
    
    #gFoldedLine =gFoldedLine/np.max(gFoldedLine,axis=0) #currently normalizaing along the path to capture the variance within a single path but not across each root or the entire image
    paddedFoldedLine = np.pad(gFoldedLine,[(0,0),(0,numIntensityParams)], "constant", constant_values = 0)
    if dilationRadius>=0:
        for pixelNum, pixel in enumerate(gFoldedLine):
            for i in range(numIntensityParams):
                x = int(pixel[0])
                y = int(pixel[1])-dilationRadius+int((i%(2*dilationRadius-1)))
                z = int(pixel[2])-dilationRadius+int(i/(2*dilationRadius-1))
                
                if x<0 or y<0 or z<0 or x>=swiData.shape[0] or y >= swiData.shape[1] or z >= swiData.shape[2]:
                    pixelIntensity = 0 #essentially pad with 0s if the shell goes out of bounds
                else:
                    pixelIntensity = swiData[x][y][z]
                paddedFoldedLine[pixelNum][3+i] = pixelIntensity
    
    paddedFoldedLine.astype(np.float)
    npFlattened = np.array(currentSplitLine).astype(np.float)
    groupArray.append(paddedFoldedLine)

In [302]:
print(bunchedPaths)

{'21 5 3': [array([[79.,  0.,  0.],
       [78.,  0.,  1.],
       [77.,  0.,  2.],
       [76.,  0.,  3.],
       [75.,  0.,  3.],
       [74.,  0.,  3.],
       [73.,  1.,  3.],
       [72.,  2.,  3.],
       [71.,  1.,  2.],
       [70.,  0.,  1.],
       [69.,  0.,  1.],
       [68.,  1.,  1.],
       [67.,  2.,  1.],
       [66.,  3.,  1.],
       [65.,  4.,  2.],
       [64.,  5.,  2.],
       [63.,  6.,  2.],
       [62.,  6.,  2.],
       [61.,  5.,  2.],
       [60.,  5.,  2.],
       [59.,  5.,  3.],
       [58.,  4.,  4.],
       [57.,  3.,  4.],
       [56.,  3.,  4.],
       [55.,  3.,  5.],
       [54.,  3.,  6.],
       [53.,  3.,  5.],
       [52.,  3.,  5.],
       [51.,  3.,  5.],
       [50.,  3.,  5.],
       [49.,  3.,  5.],
       [48.,  3.,  5.],
       [47.,  3.,  5.],
       [46.,  4.,  5.],
       [45.,  4.,  5.],
       [44.,  4.,  4.],
       [43.,  4.,  4.],
       [42.,  4.,  4.],
       [41.,  4.,  4.],
       [40.,  5.,  4.],
       [39.,  5.,  4.],
    

In [303]:
print(flatennedPaths.keys())

dict_keys(['21 5 3', '21 11 12', '21 21 1', '21 27 4', '21 40 9', '21 44 7', '21 48 3', '21 55 8', '21 76 7', '21 59 11', '21 78 10', '21 85 7', '21 98 8', '21 104 5', '21 51 12', '21 13 4', '21 29 4', '21 65 9', '21 91 13', '21 91 11', '21 20 8'])


In [304]:
#just to make sure
for keys in bunchedPaths:
    bunchedPaths[keys] = np.array(bunchedPaths[keys])#.astype(np.float)
    print(bunchedPaths[keys].shape)
    

(132, 59, 3)
(77, 59, 3)
(153, 59, 3)
(137, 59, 3)
(134, 59, 3)
(76, 59, 3)
(3, 59, 3)
(193, 59, 3)
(23, 59, 3)
(50, 59, 3)
(128, 59, 3)
(185, 59, 3)
(9, 59, 3)
(100, 59, 3)
(53, 59, 3)
(23, 59, 3)
(46, 59, 3)
(11, 59, 3)
(27, 59, 3)
(8, 59, 3)
(7, 59, 3)


In [305]:
for keys in flatennedPaths:
    flatennedPaths[keys] = np.array(flatennedPaths[keys])#.astype(np.float)
    print(flatennedPaths[keys])

[[79.  0.  0. ... 21.  5.  3.]
 [79.  1.  0. ... 21.  5.  3.]
 [79.  2.  0. ... 21.  5.  3.]
 ...
 [79.  0. 12. ... 21.  5.  3.]
 [79.  1. 12. ... 21.  5.  3.]
 [79.  4. 12. ... 21.  5.  3.]]
[[79.  3.  0. ... 21. 11. 12.]
 [79.  7.  8. ... 21. 11. 12.]
 [79.  8.  8. ... 21. 11. 12.]
 ...
 [79. 14. 14. ... 21. 11. 12.]
 [79. 15. 14. ... 21. 11. 12.]
 [79. 16. 14. ... 21. 11. 12.]]
[[79. 11.  0. ... 21. 21.  1.]
 [79. 12.  0. ... 21. 21.  1.]
 [79. 13.  0. ... 21. 21.  1.]
 ...
 [79. 22. 14. ... 21. 21.  1.]
 [79. 23. 14. ... 21. 21.  1.]
 [79. 24. 14. ... 21. 21.  1.]]
[[79. 26.  0. ... 21. 27.  4.]
 [79. 27.  0. ... 21. 27.  4.]
 [79. 28.  0. ... 21. 27.  4.]
 ...
 [79. 33. 14. ... 21. 27.  4.]
 [79. 34. 14. ... 21. 27.  4.]
 [79. 35. 14. ... 21. 27.  4.]]
[[79. 36.  0. ... 21. 40.  9.]
 [79. 37.  0. ... 21. 40.  9.]
 [79. 38.  0. ... 21. 40.  9.]
 ...
 [79. 44. 14. ... 21. 40.  9.]
 [79. 45. 14. ... 21. 40.  9.]
 [79. 46. 14. ... 21. 40.  9.]]
[[79. 47.  0. ... 21. 44.  7.]
 [79. 48.

In [306]:
clusteringMethod = DBSCAN(eps = 3, min_samples = 2) #DBSCAN(eps=0.5, min_samples=5, metric=’euclidean’, metric_params=None, algorithm=’auto’, leaf_size=30, p=None, n_jobs=None)[source]¶
labels = {}
for keys in flatennedPaths:
    clusteringMethod.fit(flatennedPaths[keys])
    labels[keys] = clusteringMethod.labels_
    #print(flatennedPaths[keys].shape)

In [307]:
print(labels)

{'21 5 3': array([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  0,  0,  3,  3,  1,  1,  1,
        2,  2,  2,  2,  0,  0,  3,  3,  3, -1,  2,  2,  2,  2,  2,  0,  0,
        3,  3,  3,  3,  2,  2,  2,  2,  4,  4,  4,  4,  4,  0,  0,  3,  3,
        3,  3,  3,  5,  5,  4,  4,  4,  4,  4,  4,  0,  0,  3,  3,  3,  3,
        5,  5,  5,  4,  4,  4,  4,  4,  4,  0,  0,  0,  3,  3,  3,  5,  5,
        5,  4,  4,  4,  4,  4,  6,  6,  6,  3,  7,  7,  5,  5,  5,  4,  4,
        4,  4,  4,  6,  6,  8,  7,  7,  7,  7,  8,  8,  8,  8,  8,  7,  8,
        8,  8,  8,  8,  9,  8,  8,  8,  8,  8,  8,  8,  9], dtype=int64), '21 11 12': array([-1,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  1,  1,  1,  0,  0,
        0,  0,  2,  1,  1,  1,  0,  0,  0,  3,  3,  4,  4,  5,  5,  2,  2,
        2,  2,  2,  0,  3,  3,  3,  3,  3,  6,  6,  5,  5,  5,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  6,  6,  6,  7,  7,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3], dtype=int64), '21 21 1': array([ 0

In [308]:
clusteringMethod = DBSCAN(eps = .1, min_samples = 2) #DBSCAN(eps=0.5, min_samples=5, metric=’euclidean’, metric_params=None, algorithm=’auto’, leaf_size=30, p=None, n_jobs=None)[source]¶
intensityLabels = {}
for keys in bunchedPaths:
    currentBunch = bunchedPaths[keys]
    currentBunch = np.reshape(currentBunch, (currentBunch.shape[0], currentBunch.shape[1]*currentBunch.shape[2])) #flatten
    currentBunch = normalize(currentBunch,norm = 'l2', axis = 0) #normalize by root node by a squared average
    pca = PCA(.95) #need to dynamically choose n_components
    
    pca.fit(currentBunch)
    print(pca.n_components_)
    transformed = pca.transform(currentBunch)
    #print(currentBunch.shape)
    clusteringMethod.fit(transformed)
    intensityLabels[keys] = clusteringMethod.labels_

3
3
3
1
3
2
2
2
2
2
1
1
1
2
1
3
1
1
3
2
3


In [309]:
print(intensityLabels)

{'21 5 3': array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3,
       3, 3, 3, 0, 0, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 0, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 4, 4, 4, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 5, 2, 2,
       2, 2, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
      dtype=int64), '21 11 12': array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  2,  2,  2,  2,  1,  1,  1,  1,
        1,  1,  1,  0,  2,  2,  2,  2,  2,  3,  3,  1,  1,  1,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  1,  1,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  2,  2], dtype=int64), '21 21 1': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
   

In [310]:
pathFiles = {}
for currentKey in intensityLabels:#currently uses the intensityLabels instead of the standard labels from clustering
    for index, cluster in enumerate(intensityLabels[currentKey]):
        filename = "".join(("pathClusters/",currentKey,";",str(cluster),".txt"))#format the filename based on the root node
        if filename in pathFiles:
            workingFile = pathFiles[filename]
        else:
            workingFile =  open(filename, "w+")
            pathFiles[filename] = workingFile
        #print(str(flatennedPaths[currentKey][cluster]))
        line = str(flatennedPaths[currentKey][index])[1:-1] #flattened paths contain the unedited paths hashed by the root node
        line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
        workingFile.write(" ".join((line,"\n")))
        

In [311]:
allPathFiles = open("pathClusters/allPathFiles.txt","w+")
for fileKeys in pathFiles:
    allPathFiles.write(pathFiles[fileKeys].name)
    allPathFiles.write("\n")
    pathFiles[fileKeys].close()
allPathFiles.write(str(clusteringMethod))
allPathFiles.close()

In [312]:
bunchFile = open("bunchedPaths.txt","w+")
for keys in bunchedPaths:
    for line in bunchedPaths[keys]:
        for pixelSet in line:
            bunchFile.write(re.sub("'|\n|[|]", "", str(pixelSet)))
            bunchFile.write("\n")
        bunchFile.write("\n")
bunchFile.close

<function TextIOWrapper.close()>

In [313]:
a = [1,2,3,4,5,6,7,8,9,10]
a =np.reshape(a,(5,2))
print(a)
#a = np.pad(a,[(0,0),(0,5)], "constant", constant_values = 0 )
#print(a)
a = a/np.max(a,axis=0)
print(a)

[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]]
[[0.11111111 0.2       ]
 [0.33333333 0.4       ]
 [0.55555556 0.6       ]
 [0.77777778 0.8       ]
 [1.         1.        ]]


In [314]:
print(swiData[68][8][1])

234.0


In [315]:
for i in range(numIntensityParams):
    y = int((i%(2*dilationRadius-1)))
    z = int(i/(2*dilationRadius-1))
    print(y,z, sep = " ")
print(numIntensityParams)

0


In [316]:
a = [[4, 1, 2, 2],
...  [1, 3, 9, 3],
...  [5, 7, 5, 1]]
b = normalize(a, norm='l1', axis=0)#l1 is a average
print(b)
pca = PCA(.9)
pca.fit(b)
c =pca.transform(b)
print(c)

[[0.4        0.09090909 0.125      0.33333333]
 [0.1        0.27272727 0.5625     0.5       ]
 [0.5        0.63636364 0.3125     0.16666667]]
[[-0.04325337  0.32364468]
 [-0.31823551 -0.1927144 ]
 [ 0.36148888 -0.13093028]]
